In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [10]:
# Load the trained model, encoders and scalers 
model = load_model('model.keras')

with open('gender_le.pkl', 'rb') as file: 
    gender_le = pickle.load(file)

with open('geography_ohe.pkl', 'rb') as file: 
    geography_ohe = pickle.load(file)

with open('scaler.pkl', 'rb') as file: 
    scaler = pickle.load(file)

In [11]:
# Sample input data
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000,
}

In [13]:
# apply ohe to geography
geo_encoded = geography_ohe.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geography_ohe.get_feature_names_out(['Geography']))
geo_encoded_df

/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
# Convert input_data into dataframe
df = pd.DataFrame([input_data])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
# Encode categorical variables
df['Gender'] = gender_le.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [16]:
# Concat ohe df and the input df
df = pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [17]:
# Scale the input data
scaled_df = scaler.transform(df)
scaled_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [18]:
# Predict churn
prediction = model.predict(scaled_df)
prediction

1/1 [==============================] - 0s 85ms/step


2025-09-06 16:16:13.478327: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


array([[0.03677541]], dtype=float32)

In [19]:
prediction_probability = prediction[0][0]
prediction_probability

0.03677541

In [20]:
if prediction_probability > 0.5: 
    print("Customer is likely to churn.")
else: 
    print("Customer is not likely to churn.")

Customer is not likely to churn.
